In [5]:
import pathlib
import math

from compas.files import OBJ
from compas.geometry import Frame, Rotation
from compas.datastructures import Mesh
from compas_model.model import Model
from compas_model.elements import BlockElement

filepath = pathlib.Path().home() / "Code" / "compas_model" / "data" / "wall_stack.obj"

obj = OBJ(filepath)
obj.read()

meshes = []
for name in obj.objects:  # type: ignore
    vertices, faces = obj.objects[name]  # type: ignore
    mesh = Mesh.from_vertices_and_faces(vertices, faces)
    mesh.name = name
    meshes.append(mesh)

model = Model()

supports = model.add_group(name="supports")
blocks = model.add_group(name="blocks")

block = BlockElement(shape=meshes[0], is_support=True)
model.add_element(block, parent=supports)

for mesh in meshes[1:]:
    block = BlockElement(shape=mesh)
    model.add_element(block, parent=blocks)

R = Rotation.from_axis_and_angle(axis=[0, 0, 1], angle=math.radians(30))

blocks.frame = Frame.worldXY()
blocks.frame.transform(R)

blocks.children[-1].element.transformation = R


In [6]:
from compas_model.algorithms import collider

interfaces = []
for item in collider.get_collision_pairs(model, tolerance_area=0.1):
    i, j, collisions = item
    for faces, polygon in collisions:
        interfaces.append(polygon)
        model.add_interaction(model.elementlist[i], model.elementlist[j])

In [7]:
model.print()

Spatial Hierarchy
└── <TreeNode: root>
    ├── <TreeNode: supports>
    │   └── <TreeNode>
    └── <TreeNode: blocks>
        ├── <TreeNode>
        ├── <TreeNode>
        ├── <TreeNode>
        ├── <TreeNode>
        └── <TreeNode>
Element Interactions
0
- 1: None
1
- 0: None
- 2: None
2
- 1: None
- 3: None
3
- 2: None
- 4: None
4
- 3: None
- 5: None
5
- 4: None
Element Groups
n/a


In [8]:
from compas_notebook.viewer import Viewer
from compas.colors import Color

viewer = Viewer()

viewer.scene.add(model,
    show_elements=True,
    show_element_faces=False,
    show_faces=False,
    show_edges=True,
    show_vertices=True,
    vertexsize=0.1
)

# for element in model.elementlist:
#     viewer.scene.add(element, color=Color.from_hex('#cccccc'), show_faces=False, show_edges=True)

for polygon in interfaces:
    viewer.scene.add(polygon, color=Color.red())

viewer.show()